In [1]:
import anndata
import numpy as np
import scanpy as sc
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
dataset = 'Erythroid_Human'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')
adata.obs['clusters'] = adata.obs['type2'].to_numpy()

In [ ]:
#vv.preprocess(adata, n_gene=2000, min_shared_counts=20)
#adata.write_h5ad(f'data/{dataset}_pp.h5ad')

In [9]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/discrete/{dataset}'
gene_plot = ['CNN3','CYR61','ABCG2','HBA2']

# Discrete VeloVAE

In [10]:
figure_path = f'{figure_path_base}/VeloVAE'
model_path = f'{model_path_base}/VeloVAE'

torch.manual_seed(2022)
np.random.seed(2022)
dvae = vv.DVAE(adata, 
               tmax=20, 
               dim_z=5, 
               device='cuda:0',
               init_method='random')

dvae.train(adata, gene_plot=gene_plot, plot=False, figure_path=figure_path)
dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dvae', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Random Initialization.
Gaussian Prior.
Library scale (U): Max=4.62, Min=0.04, Mean=0.90
Library scale (S): Max=5.19, Min=0.14, Mean=0.98
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 197, test iteration: 392
*********       Stage 1: Early Stop Triggered at epoch 325.       *********
*********                      Stage  2                       *********


  0%|          | 0/35877 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.007
Average Set Size: 853
*********       Stage 2: Early Stop Triggered at epoch 467.       *********
*********              Finished. Total Time =   0 h : 21 m : 51 s             *********
Final: Train ELBO = -2299.200,           Test ELBO = -2345.462
       Training MSE = 32.130, Test MSE = 51.681


# Discrete Full VB

In [11]:
figure_path = f'{figure_path_base}/FullVB'
model_path = f'{model_path_base}/FullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.DVAEFullVB(adata, 
                       tmax=20, 
                       dim_z=5, 
                       device='cuda:0',
                       init_method='random')

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'dfullvb', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Random Initialization.
Gaussian Prior.
Library scale (U): Max=4.62, Min=0.04, Mean=0.90
Library scale (S): Max=5.19, Min=0.14, Mean=0.98
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 197, test iteration: 392
*********       Stage 1: Early Stop Triggered at epoch 478.       *********
*********                      Stage  2                       *********


  0%|          | 0/35877 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.012
Average Set Size: 856
*********       Stage 2: Early Stop Triggered at epoch 614.       *********
*********              Finished. Total Time =   0 h : 32 m : 15 s             *********
Final: Train ELBO = -2304.150,           Test ELBO = -2339.753
       Training MSE = 29.743, Test MSE = 47.523


# Evaluation

In [ ]:
cluster_edges = [('MEMP', 'Early Erythroid'), 
                 ('Early Erythroid', 'Mid  Erythroid'), 
                 ('Mid  Erythroid', 'Late Erythroid')]
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE','Discrete FullVB'],
                 ['dvae','dfullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)